In [16]:
from ctypes import sizeof
from tabnanny import verbose
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.operators import PolynomialTensor
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from pyscf.lo import Boys
from pyscf.lo import PM
from qiskit_nature.units import DistanceUnit
import numpy as np  
from pyscf.tools import molden

In [17]:
def h6_linear_chain_loc(dist=0.75):
    # Run HF with orbital localization
    driver = PySCFDriver(
        atom='H 0.0 0.0 0.0; H 0.0 0.0 0.75; H 0.0 0.00 1.50; H 0.0  0.00 2.25; H 0.0 0.00 3.00; H 0.0 0.00 3.75',
        basis="sto3g",
        unit=DistanceUnit.ANGSTROM
    )
    problem = driver.run()
    mol = driver._mol
    mf = driver._calc
    mo = mf.mo_coeff
    hamiltonian = problem.hamiltonian
    nuclear_repulsion = problem.nuclear_repulsion_energy
    hamiltonian.electronic_integrals.alpha += PolynomialTensor({"": nuclear_repulsion})
    hamiltonian.nuclear_repulsion_energy = None
    
    # Jordan-Wigner mapping 
    mapper = JordanWignerMapper()
    # qubit_op = mapper.map(hamiltonian.second_q_op())
    tapered_mapper = problem.get_tapered_mapper(mapper)
    qubit_op = tapered_mapper.map(hamiltonian.second_q_op())
    
    return qubit_op
h6_linear_op_equi=h6_linear_chain_loc(0.75)
eigenvalues, eigenvectors = np.linalg.eig(h6_linear_op_equi.to_matrix())
print(np.min(eigenvalues))


(-3.1553048004765167-3.9429660226413597e-31j)


In [18]:
h6_linear_op_equi

SparsePauliOp(['IIIIIIIII', 'ZIZIZIZIZ', 'IIIIIZZZX', 'ZIZIZZIZX', 'IIIIIZXII', 'ZIZIZZXIZ', 'ZIZIZZIZI', 'ZIZIZZIXZ', 'IIIIIIIXZ', 'ZIZIZXZIZ', 'IIIIIXZZZ', 'IIIIIIIIZ', 'IIIIIIYZY', 'IIIIIIXZX', 'IIIIIIIZI', 'IIIIIYZYI', 'IIIIIXZXI', 'IIIIIIZII', 'IIIIIZIII', 'ZZZZZIIII', 'ZZZXIIIII', 'IIIXZIIII', 'ZXIIIIIII', 'IXZZZIIII', 'IIIIZIIII', 'IIYZYIIII', 'IIXZXIIII', 'YZZZYIIII', 'XZZZXIIII', 'IIIZIIIII', 'IYZYIIIII', 'IXZXIIIII', 'IIZIIIIII', 'YZYIIIIII', 'XZXIIIIII', 'IZIIIIIII', 'ZIIIIIIII', 'IIIIIZZZZ', 'IIIIIZZXI', 'ZIZIZIZXI', 'IIIIIXIII', 'ZIZIZXIZI', 'ZIZIZIZII', 'ZIZIZIXZX', 'ZIZIZIYZY', 'ZIZIZIZZZ', 'ZIZIZYIYZ', 'ZIZIZXIXZ', 'ZIZIZIIIZ', 'ZIZIZZZIZ', 'IZIZIIZIZ', 'IZIXZIZIZ', 'ZIZXIIZIZ', 'IXZIZIZIZ', 'ZXIZIIZIZ', 'ZIZIIIZIZ', 'ZIXZXIZIZ', 'ZIYZYIZIZ', 'XZIZXIZIZ', 'YZIZYIZIZ', 'ZIZZZIZIZ', 'ZYIYZIZIZ', 'ZXIXZIZIZ', 'ZIIIZIZIZ', 'XZXIZIZIZ', 'YZYIZIZIZ', 'ZZZIZIZIZ', 'IIZIZIZIZ', 'ZIZIZIZIX', 'IIIIIIIIX', 'ZIZIZIXZI', 'IIIIIIXZZ', 'IIIIIZZXX', 'IIIIIZZYY', 'ZIZIZIZYY', 'ZIZIZZIYY

In [19]:
print(np.sort(eigenvalues))
np.save("h6_linear_op_equi.npy", h6_linear_op_equi)

[-3.15530480e+00-3.94296602e-31j -2.55997948e+00+5.08509926e-32j
 -2.50220668e+00-3.12011425e-31j -2.21316786e+00+2.00657564e-30j
 -2.17026967e+00-3.94424945e-30j -2.01267274e+00+5.33422466e-20j
 -2.00116070e+00+7.26987425e-17j -1.93515989e+00+4.24515036e-17j
 -1.85442956e+00-2.77583869e-17j -1.85442956e+00+2.22485062e-16j
 -1.85442956e+00-1.05180648e-16j -1.72486924e+00+7.66245570e-17j
 -1.70394229e+00-2.32452946e-16j -1.70394229e+00+1.66532923e-16j
 -1.67568778e+00+4.25975678e-17j -1.63363422e+00-1.64199219e-16j
 -1.50091094e+00+3.44924717e-16j -1.45279733e+00-1.12753472e-17j
 -1.45279733e+00+1.71357719e-17j -1.44816360e+00-2.31380276e-17j
 -1.40596794e+00+1.09720627e-16j -1.37366858e+00-6.61455344e-16j
 -1.37366858e+00+1.38777874e-17j -1.37366858e+00-4.44177301e-16j
 -1.35068057e+00+5.37855823e-16j -1.31471640e+00-1.10310562e-16j
 -1.27979600e+00-1.75769236e-17j -1.27979600e+00-9.99684620e-17j
 -1.24017764e+00-5.68143215e-17j -1.22210785e+00+1.91172198e-17j
 -1.14568145e+00+1.537986

In [20]:
def h6_linear_chain_loc():
    # Run HF with orbital localization
    driver = PySCFDriver(
        atom='H 0.0 0.0 0.0; H 0.0 0.0 1.5; H 0.0 0.00 3.00; H 0.0  0.00 4.50; H 0.0 0.00 6.00; H 0.0 0.00 7.50',
        basis="sto3g",
        unit=DistanceUnit.ANGSTROM
    )
    problem = driver.run()
    mol = driver._mol
    mf = driver._calc
    mo = mf.mo_coeff
    hamiltonian = problem.hamiltonian
    nuclear_repulsion = problem.nuclear_repulsion_energy
    hamiltonian.electronic_integrals.alpha += PolynomialTensor({"": nuclear_repulsion})
    hamiltonian.nuclear_repulsion_energy = None
    
    # Jordan-Wigner mapping 
    mapper = JordanWignerMapper()
    # qubit_op = mapper.map(hamiltonian.second_q_op())
    tapered_mapper = problem.get_tapered_mapper(mapper)
    qubit_op = tapered_mapper.map(hamiltonian.second_q_op())
    
    return qubit_op
h4_linear_op_stretched=h6_linear_chain_loc()
eigenvalues, eigenvectors = np.linalg.eig(h4_linear_op_stretched.to_matrix())
print(np.min(eigenvalues))


(-2.995565425831932+0j)


In [21]:
np.save("h6_linear_op_stretched.npy", h6_linear_op_stretched)